In [1]:
using Revise

In [1]:
include("detector.jl")
initialize(cutoff=0.85, ngram=5)
sum(length.(PROFILES))/length(PROFILES)

┌ Info: zho: cutoff(0.85) not reached. current: 0.8238682009344496. vocab size: 73243
└ @ Main /home/guoyongzhi/langid_expirement/detector.jl:40


9758.62

In [16]:
function langprob(text; candidates=5, ngram=NGRAM, languages=LANGUAGES, profiles=PROFILES)
    p = count_one_to_ngrams(text, ngram)
    vs = sum(values(p))
    map!(v -> v / vs, values(p))
    lls = loglikelihood.(Ref(p), profiles)
    ls = exp.(lls)
    ls = ls ./ sum(ls)
    si = sortperm(ls, rev=true)[1:candidates]
    [k => v for (k, v) in zip(languages[si], ls[si])]
end


10-element Vector{Pair{String, Float64}}:
 "zho" => 0.62924682635058
 "jpn" => 0.05244491733791451
 "kor" => 0.012870047024445788
 "heb" => 0.012096720179166935
 "hin" => 0.011165821665276653
 "ben" => 0.011105808442047563
 "mar" => 0.010937988980142262
 "yid" => 0.010766279173229682
 "fas" => 0.008894975618573728
 "vie" => 0.008826884212367854

In [141]:
x, y = WV[rand(1:length(WV))]
y, langprob(x, candidates=5, ngram=7)

("jpn", ["jpn" => 0.31106811055831624, "zho" => 0.0552208262531225, "kor" => 0.021691741681780025, "tat" => 0.017003364173065423, "ara" => 0.016890944932712364])

In [11]:
# initialize(cutoff=0.85, ngram=5)
langprob("good", ngram=4)

5-element Vector{Pair{String, Float64}}:
 "nds" => 0.135495124819153
 "eng" => 0.13288222115463774
 "nld" => 0.0624004906963523
 "pol" => 0.036721264379596975
 "dan" => 0.03586694581870948

In [19]:
include("benchmark.jl")
include("datasetloader.jl")
WV = WikiDataSet("corpus/wikipedia/test", langs=ALL_LANGUAGES)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=ALL_LANGUAGES)

521469-element TatoebaDataset:
             "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                          "Mi veturigos vin al la flughaveno." => "epo"
                                                                 "Kio mankas?" => "epo"
                                            "Aĥ, kiam ili renkontiĝos denove?" => "epo"
 "Kiam mi estis infano, mi pasigis multajn horojn legante sola en mia ĉambro." => "epo"
                                              "Ŝajnas, ke la trajno malfruas." => "epo"
                                                                "Venu rapide!" => "epo"
                                                   "Li montris al mi albumon." => "epo"
                                                             "Estis du kukoj." => "epo"
                                      "Mia filo scipovas jam nombri ĝis cent." => "epo"
                                                                               ⋮
        

In [6]:
@time benchmark(
    # "likelihood-n5"=>detector_lh, 
    "n4"=>t->langid(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=TV, languages=ALL_LANGUAGES[14:18])

n4 32.421179 seconds (13.15 M allocations: 959.471 MiB, 0.76% gc time, 0.65% compilation time)


 32.840781 seconds (13.65 M allocations: 993.415 MiB, 0.75% gc time, 1.84% compilation time)


1-element Vector{Any}:
 "n4" => [0.9739471805852962, 0.9853345554537122, 0.46324143692564745, 0.9915330212172527, 0.7335958005249343]

In [33]:
using StatProfilerHTML

In [34]:

@profilehtml benchmark(
    # "likelihood-n5"=>detector_lh, 
    "likelihood-n4"=>t->detector_lh2(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=TV, languages=LANGS[14:18])

likelihood-n4

 25.162121 seconds (13.11 M allocations: 1.018 GiB, 0.78% gc time, 0.02% compilation time)


┌ Info: Wrote profiling output to file://c:\Users\momos\repositories\langid_expirement\statprof/index.html .
└ @ StatProfilerHTML C:\Users\momos\.julia\packages\StatProfilerHTML\DKdvQ\src\StatProfilerHTML.jl:28


1-element Vector{Any}:
 "likelihood-n4" => [0.9829764453961456, 0.9871677360219981, 0.16040100250626566, 0.9901384600059767, 0.8805774278215223]

In [3]:
using JET

In [10]:
WB = benchmark(
    # "likelihood-n5"=>detector_lh, 
    "likelihood-n4"=>t->detector_lh(t, ngram=4),
    "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=LANGS)

likelihood-n4

 81.693988 seconds (73.70 M allocations: 5.453 GiB, 0.99% gc time, 0.11% compilation time)
likelihood-n3

 39.605433 seconds (55.49 M allocations: 3.933 GiB, 1.32% gc time, 0.19% compilation time)


2-element Vector{Any}:
 "likelihood-n4" => [1.0, 0.99, 1.0, 1.0, 1.0, 0.985, 0.995, 0.965, 1.0, 1.0  …  1.0, 0.99, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.985, 0.995]
 "likelihood-n3" => [1.0, 0.99, 1.0, 0.98, 0.995, 0.985, 0.985, 0.96, 1.0, 1.0  …  1.0, 0.985, 0.98, 0.995, 0.985, 0.985, 1.0, 0.965, 0.975, 0.995]

In [5]:
WB = benchmark(
    # "likelihood-n5"=>detector_lh, 
    "likelihood-n4"=>t->detector_lh(t, ngram=4),
    "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=LANGS)

likelihood-n4

127.426978 seconds (1.63 G allocations: 28.846 GiB, 4.42% gc time, 0.59% compilation time)


likelihood-n3

 76.725221 seconds (885.37 M allocations: 16.355 GiB, 4.33% gc time, 0.20% compilation time)


2-element Vector{Any}:
 "likelihood-n4" => [1.0, 0.99, 1.0, 1.0, 1.0, 0.985, 0.995, 0.965, 1.0, 1.0  …  1.0, 0.99, 0.985, 0.995, 0.985, 0.985, 1.0, 0.965, 0.985, 0.995]
 "likelihood-n3" => [1.0, 0.99, 1.0, 0.98, 0.995, 0.985, 0.985, 0.96, 1.0, 1.0  …  1.0, 0.985, 0.98, 0.995, 0.985, 0.985, 1.0, 0.965, 0.975, 0.995]

In [6]:
showtable(WB, LANGS, threshold=-0.5)

|               | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likeliho

|               | Average | ara     | bel    | ben     | bul     | cat     | ces    | dan    | deu    | ell     | eng     | epo     | fas     | fin    | fra     | hau    | hbs    | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor     | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus     | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|---------|---------|--------|--------|--------|---------|---------|---------|---------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|---------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likelihood-n4 |  97.63% | 100.00% | 99.00% | 100.00% | 100.00% | 100.00% | 98.50% | 99.50% | 96.50% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.50% | 66.50% | 100.00% | 90.00% |  99.50% | 93.00% | 93.00% | 98.50% | 99.00% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 95.00% | 98.00% | 99.00% | 99.00% | 97.50% | 93.50% |  99.50% | 99.50% | 91.00% | 100.00% | 96.50% | 100.00% | 99.00% | 98.50% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 98.50% | 99.50% |
| likelihood-n3 |  95.93% | 100.00% | 99.00% | 100.00% |  98.00% |  99.50% | 98.50% | 98.50% | 96.00% | 100.00% | 100.00% | 100.00% | 100.00% | 99.00% | 100.00% | 98.00% | 10.50% | 100.00% | 89.50% | 100.00% | 88.50% | 84.50% | 98.50% | 98.50% | 99.00% | 99.00% | 100.00% | 99.00% | 97.00% | 100.00% | 92.00% | 97.00% | 99.00% | 98.50% | 96.50% | 91.00% | 100.00% | 99.50% | 90.00% |  99.50% | 96.50% | 100.00% | 98.50% | 98.00% | 99.50% | 98.50% | 98.50% | 100.00% | 96.50% | 97.50% | 99.50% |


In [7]:
WB2 = benchmark(
    "likelihood-n5"=>detector_lh, 
    # "likelihood-n4"=>t->detector_lh(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=("mkd", "hbs"))

likelihood-n5

  9.023298 seconds (90.31 M allocations: 1.666 GiB, 3.74% gc time, 1.14% compilation time)


1-element Vector{Any}:
 "likelihood-n5" => [0.99, 0.88]

In [8]:
WB2 = benchmark(
    "likelihood-n5"=>detector_lh, 
    # "likelihood-n4"=>t->detector_lh(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=TV, languages=("mkd", "hbs"))

likelihood-n5

 32.151567 seconds (117.73 M allocations: 2.000 GiB, 1.17% gc time, 0.05% compilation time)


1-element Vector{Any}:
 "likelihood-n5" => [0.9469910371318822, 0.21888053467000834]